In [1]:
import warnings    
warnings.simplefilter("ignore", UserWarning)
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt    
from sklearn.linear_model import LinearRegression, Ridge, Lasso, SGDRegressor
from sklearn.preprocessing import PolynomialFeatures, FunctionTransformer,\
    OneHotEncoder, LabelEncoder, PowerTransformer
from sklearn.neighbors import KNeighborsRegressor 
import seaborn as sns
from sklearn.impute import KNNImputer
import re
from sklearn.tree import DecisionTreeRegressor
from scipy import stats  
from sklearn.compose import ColumnTransformer 
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import cross_val_score
import pickle 
from src.utils import previous_owners, convert_text_to_number,\
    passengers_number, get_model_scores, outlier_detector,\
    outlier_detector_zscore 

Reading Data

In [2]:
data = pd.read_csv('../data/cars_data.csv')